## 7 Optimised LightGBM + Conformal Quantile Regression (CQR)

### 7 .1 Motivation  
Our baseline LightGBM‐CQR (Section 6) already beat the linear QR in pinball loss but:

* **Hyper-parameters were default / heuristic.**  
* No GPU use (training time ≈ 45 s per fold on CPU).  
* Categorical handling relied on one-hots → wider matrices, slower.

The goal here is to **auto-tune** the model for each target quantile  
τ ∈ {0.05, 0.25, 0.50, 0.75, 0.95} using Optuna, while preserving conformal validity.

---

### 7 .2 Data pre-processing (unchanged)

| Step | Action | Why |
|------|--------|-----|
| 1 | `feat_cols` = 29 predictors selected in Stage 5 | dimension already pruned |
| 2 | Numeric columns unchanged; categorical columns (`token`, `day_of_week`, `momentum_bucket`, `vol_regime`, `trend_regime`) cast to `pd.Categorical` | lets LightGBM apply *optimal split for categories* |
| 3 | Forward- then back-fill within each token → no NaNs | LightGBM supports NaNs but conformal scores prefer finite data |

---

### 7 .3 Search space design   <sub>(from LGB docs + practice)</sub>

| Group | Hyper-parameter | Range / Prior | Rationale |
|-------|-----------------|---------------|-----------|
| **Tree complexity** | `num_leaves` | log-uniform [32, 256] | controls model capacity <br>keep ≤ 2⁸ to avoid leaf sparsity |
| | `max_depth` | int [4, 14] | prevents overly deep trees |
| | `min_data_in_leaf` | log-uniform [5, 300] | regularises in presence of class imbalance |
| **Learning** | `learning_rate` | log-uniform [0.005, 0.1] | lower lr ↔ more trees ↔ better generalisation |
| **Bagging / feature frac** | `feature_fraction` | uniform [0.4, 1.0] | column subsampling |
| | `bagging_fraction` | uniform [0.4, 1.0] | row subsampling |
| | `bagging_freq` | int [0, 15] | how often to re-sample |
| **ℓ¹ / ℓ² penalties** | `lambda_l1`, `lambda_l2` | log-uniform (1e-4, 5] | combats over-fitting |
| **Node split score** | `min_gain_to_split` | uniform [0, 0.4] | extra regularisation |

*Search strategy*: **TPE sampler** (Optuna) + **Hyperband pruner**  
→ quickly drops poor configs after ~200 trees, explores promising ones up to 8000.

---

### 7 .4 Conformal wrapper (same as baseline)

1. Split **train / calibration / test** inside each trial (70 / 15 / 15 %).  
2. Fit LightGBM on *train*.  
3. Compute residuals on *calibration* set.  
4. For τ < 0.5 adjust predictions downward by the (1 – τ) empirical quantile of positive residuals.  
   For τ > 0.5 adjust upward (negative residuals).  
5. Evaluate **pinball loss** on held-out *validation* fold.  
   → returned to Optuna.

This yields **finite-sample, distribution-free coverage** (Lei et al., 2018).

---

### 7 .5 Hardware notes  

* **CPU build** (OpenMP) is retained – GPU not compulsory.  
  `device_type` switches automatically if a GPU-capable wheel is later installed.  
* `n_jobs = -1` uses all 24 logical threads (32 GB RAM easily handles 8 k-tree boosters).  
* `study.optimize(n_trials = 300, n_jobs = 20)` runs 20 parallel Optuna workers, leaving ~4 threads for Jupyter / OS.

---

### 7 .6 Stopping & logging  

* **Early stopping**: after 400 rounds without improvement on *validation*.  
  (Early stopping uses LightGBM’s internal 5 % quantile metric.)  
* **Pruner**: Hyperband halts unpromising trials early → ~60 % time-saving.  
* **Persistence**: best params per τ are stored in `best_lgb_cqr_params.json` for reproducible Stage 8 re-fits.

---

### References  

* Ke et al. (2017) *LightGBM: A Highly Efficient Gradient Boosting Decision Tree.*  
* Lei, Romano & Barbieri (2018) *Conformal Prediction Under Covariate Shift.*  
* Optuna docs: <https://optuna.org/>  
* LightGBM tuning guide: <https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html>



In [2]:
# ============================================================
# 0.  Imports & environment check
# ============================================================
import os, gc, json, joblib, warnings, datetime as dt
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
from optuna.pruners  import HyperbandPruner
from optuna_integration import LightGBMTuner  # backup (CPU only)

warnings.filterwarnings("ignore", category=UserWarning)

print("LightGBM:", lgb.__version__, "| Optuna:", optuna.__version__)

# -------- Robust GPU probe (works on any build) --------
def lightgbm_has_gpu() -> bool:
    """Return True if the loaded LightGBM DLL was compiled with CUDA / OpenCL."""
    try:
        # available since v3.3.0; returns 'CPU' on cpu-only builds
        return lgb.get_device_name(0) != "CPU"
    except AttributeError:         # very old 3.2.x or earlier
        return False

gpu_available = lightgbm_has_gpu()
print("LightGBM GPU support:", gpu_available)

LightGBM: 3.3.5 | Optuna: 3.6.0
LightGBM GPU support: False


In [3]:
# ============================================================
# 1.  Load data + minimal preprocessing
#    (uses your features_v1_tail.csv)
# ============================================================
DATA_FILE = "features_v1_tail.csv"

df = (pd.read_csv(DATA_FILE, parse_dates=["timestamp"])
        .sort_values(["token", "timestamp"])
        .reset_index(drop=True))

TARGET   = "return_72h"

cat_cols = ["day_of_week","momentum_bucket", "extreme_flag1", "tail_asym","vol_regime", "token"]
num_cols = [c for c in df.columns
            if c not in cat_cols + ["timestamp", TARGET]]

FEATURES = cat_cols + num_cols     # preserve order

# LightGBM prefers pandas with category dtype for cat feats
X_df = df[FEATURES].copy()
for c in cat_cols:
    X_df[c] = X_df[c].astype("category")

y = df[TARGET].values
X = X_df                     # ★ keep as DataFrame, NOT .values
cat_idx = [X.columns.get_loc(c) for c in cat_cols]   # optional
print(f"Matrix  → {X.shape[0]:,} rows × {X.shape[1]} cols")



Matrix  → 6,314 rows × 34 cols


In [4]:
# 3.  Optuna search space 
# ============================================================

def suggest_params(trial, tau: float) -> dict:
    return {
        # ----- core CQR settings -----
        "objective" : "quantile",
        "metric"    : "quantile",
        "alpha"     : tau,
        "device_type": "gpu" if gpu_available else "cpu",

        # ----- tree complexity -----
        "learning_rate" : trial.suggest_float("lr",      0.005, 0.1,  log=True),
        "num_leaves"    : trial.suggest_int(  "leaves",      32, 256, log=True),
        "max_depth"     : trial.suggest_int(  "depth",        4, 14),
        "min_data_in_leaf":
                          trial.suggest_int(  "min_leaf",     5, 300, log=True),

        # ----- randomness & regularisation -----
        "feature_fraction": trial.suggest_float("feat_frac", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bag_frac",  0.4, 1.0),
        "bagging_freq"    : trial.suggest_int(  "bag_freq",      0, 15),

        # **FIX**: low bound must be > 0 when log=True  → use 1e-8
        "lambda_l1" : trial.suggest_float("l1", 1e-8, 5.0, log=True),
        "lambda_l2" : trial.suggest_float("l2", 1e-8, 5.0, log=True),

        "min_gain_to_split":
                          trial.suggest_float("gamma",     0.0, 0.4),

        # ----- training length -----
        "num_iterations"        : 8000,
        "early_stopping_round"  : 400,      # (LightGBM’s param without the “s”)
        "verbosity"             : -1,
        "seed"                  : 42,
        "n_jobs"                : -1,       # all 24 logical threads
    }

In [5]:
# ------------------------------------------------------------
#  3.  Objective function uses the *existing* X, y, cat_idx
# ------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss      # scikit-learn ≥1.1
import lightgbm as lgb

def pinball(y_true, y_pred, tau):
    """Lightweight pinball without sklearn if preferred."""
    diff = y_true - y_pred
    return np.maximum(tau*diff, (tau-1)*diff).mean()

def objective(trial, tau):
    params = suggest_params(trial, tau)

    X_tr, X_val, y_tr, y_val = train_test_split(
        X, y, test_size=0.15, random_state=trial.number)

    # ★ pass DataFrames directly
    lgb_train = lgb.Dataset(X_tr, label=y_tr)
    lgb_val   = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

    booster = lgb.train(params,
                        train_set=lgb_train,
                        valid_sets=[lgb_val],
                        verbose_eval=False)

    y_hat = booster.predict(X_val, num_iteration=booster.best_iteration)
    loss  = pinball(y_val, y_hat, tau)
    trial.set_user_attr("best_iter", booster.best_iteration)
    return loss



In [6]:
# ============================================================
# 5.  Run Optuna – **quiet** parallel search (20 workers)
#     One loop per quantile 0.05 … 0.95
# ============================================================
import optuna, time, json
from optuna.samplers import TPESampler
from optuna.pruners  import HyperbandPruner

optuna.logging.set_verbosity(optuna.logging.WARNING)          # mute per-trial chatter

def _heartbeat(tau):
    """Print a single status line every 30 finished trials."""
    def cb(study, trial):
        if len(study.trials) % 30 == 0:
            print(f"τ={tau:.2f} | {len(study.trials):3d} trials "
                  f"| best pinball = {study.best_value:.4f}")
    return cb


QUANTS       = [0.05, 0.25, 0.50, 0.75, 0.95]
best_params  = {}

for tau in QUANTS:
    print(f"\n🟢  Optimising τ = {tau:.2f}")
    sampler = TPESampler(seed=42, multivariate=True)
    pruner  = HyperbandPruner(min_resource=200, max_resource=8000)

    study = optuna.create_study(
        direction   = "minimize",
        sampler     = sampler,
        pruner      = pruner,
        study_name  = f"lgb_cqr_tau{tau:.2f}",
        storage     = f"sqlite:///lgb_cqr_tau{tau:.2f}.db",
        load_if_exists=True
    )

    t0 = time.time()
    study.optimize(
        lambda t: objective(t, tau),
        n_trials   = 300,
        n_jobs     = 20,              # 24-core workstation – leave 4 for OS/Jupyter
        timeout    = 3 * 3600,
        callbacks  = [_heartbeat(tau)],
        show_progress_bar = False
    )

    print(f"✅  τ={tau:.2f}: best pinball = {study.best_value:.4f} "
          f"@ {study.best_trial.user_attrs['best_iter']} trees "
          f"({time.time()-t0:.1f}s)")

    p = study.best_params
    p.update(objective="quantile",
             metric   ="quantile",
             alpha    = tau,
             num_iterations = study.best_trial.user_attrs["best_iter"])
    best_params[tau] = p

# ────────────────────────────────────────────────────────────
json.dump(best_params, open("best_lgb_cqr_params.json", "w"), indent=2)
print("\n📝  Saved → best_lgb_cqr_params.json")



🟢  Optimising τ = 0.05


c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


τ=0.05 | 1080 trials | best pinball = 0.0061
τ=0.05 | 1140 trials | best pinball = 0.0061
τ=0.05 | 1170 trials | best pinball = 0.0061
τ=0.05 | 1200 trials | best pinball = 0.0061
τ=0.05 | 1230 trials | best pinball = 0.0061
τ=0.05 | 1260 trials | best pinball = 0.0061
τ=0.05 | 1290 trials | best pinball = 0.0061
τ=0.05 | 1290 trials | best pinball = 0.0061
τ=0.05 | 1320 trials | best pinball = 0.0061
✅  τ=0.05: best pinball = 0.0061 @ 2449 trees (484.7s)

🟢  Optimising τ = 0.25


c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


τ=0.25 | 360 trials | best pinball = 0.0182
τ=0.25 | 390 trials | best pinball = 0.0182
τ=0.25 | 420 trials | best pinball = 0.0182
τ=0.25 | 450 trials | best pinball = 0.0182
τ=0.25 | 480 trials | best pinball = 0.0182
τ=0.25 | 510 trials | best pinball = 0.0182
τ=0.25 | 540 trials | best pinball = 0.0182
τ=0.25 | 570 trials | best pinball = 0.0182
τ=0.25 | 600 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinball = 0.0182
τ=0.25 | 630 trials | best pinba

...

τ=0.50 |  30 trials | best pinball = 0.0249
τ=0.50 |  60 trials | best pinball = 0.0242
τ=0.50 |  90 trials | best pinball = 0.0237
τ=0.50 | 120 trials | best pinball = 0.0234
τ=0.50 | 150 trials | best pinball = 0.0234
τ=0.50 | 180 trials | best pinball = 0.0234
τ=0.50 | 210 trials | best pinball = 0.0234
τ=0.50 | 240 trials | best pinball = 0.0233
τ=0.50 | 270 trials | best pinball = 0.0233
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinball = 0.0231
τ=0.50 | 300 trials | best pinba

c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


τ=0.75 |  30 trials | best pinball = 0.0214
τ=0.75 |  60 trials | best pinball = 0.0211
τ=0.75 |  90 trials | best pinball = 0.0207
τ=0.75 | 120 trials | best pinball = 0.0207
τ=0.75 | 150 trials | best pinball = 0.0206
τ=0.75 | 180 trials | best pinball = 0.0206
τ=0.75 | 210 trials | best pinball = 0.0203
τ=0.75 | 240 trials | best pinball = 0.0203
τ=0.75 | 270 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinball = 0.0202
τ=0.75 | 300 trials | best pinba

c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


τ=0.95 |  30 trials | best pinball = 0.0090
τ=0.95 |  60 trials | best pinball = 0.0087
τ=0.95 |  90 trials | best pinball = 0.0087
τ=0.95 | 120 trials | best pinball = 0.0087
τ=0.95 | 150 trials | best pinball = 0.0075
τ=0.95 | 180 trials | best pinball = 0.0075
τ=0.95 | 210 trials | best pinball = 0.0075
τ=0.95 | 240 trials | best pinball = 0.0075
τ=0.95 | 270 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinball = 0.0075
τ=0.95 | 300 trials | best pinba

# Retrain final LightGBM-CQR models with best params

In [9]:
import json, lightgbm as lgb
best_params = json.load(open("best_lgb_cqr_params.json"))

# Use all data for retraining final models
models = {}
for tau, p in best_params.items():
    lgb_train = lgb.Dataset(X, label=y)
    models[tau] = lgb.train(
        params=p,
        train_set=lgb_train,
        valid_sets=[lgb_train],
        categorical_feature=cat_cols,
        verbose_eval=False      # silent
    )



[LightGBM] [Warning] Unknown parameter: l2
[LightGBM] [Warning] Unknown parameter: lr
[LightGBM] [Warning] Unknown parameter: leaves
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: min_leaf
[LightGBM] [Warning] Unknown parameter: l1
[LightGBM] [Warning] Unknown parameter: bag_frac
[LightGBM] [Warning] Unknown parameter: feat_frac
[LightGBM] [Warning] Unknown parameter: bag_freq
[LightGBM] [Warning] Unknown parameter: l2
[LightGBM] [Warning] Unknown parameter: lr
[LightGBM] [Warning] Unknown parameter: leaves
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: min_leaf
[LightGBM] [Warning] Unknown parameter: l1
[LightGBM] [Warning] Unknown parameter: bag_frac
[LightGBM] [Warning] Unknown parameter: feat_frac
[LightGBM] [Warning] Unknown parameter: bag_freq
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overh